### 중요한 이메일이 오면 윈도우 알람으로 알려주기

In [4]:
import imaplib
import email
from email import policy 

1.환경 변수에 값 설정한 후 해당 값 가져오기 

In [2]:
import os
# 환경 변수의 값을 가져오기
value = os.environ.get('EMAIL_PASSWORD')

if value:
    print(f'환경 변수 값이 존재합니다.')
else:
    print('해당 환경 변수가 존재하지 않습니다.')

환경 변수 값이 존재합니다.


In [13]:
imap = imaplib.IMAP4_SSL('imap.naver.com')
id = 'kktpsh'
pw = value          # 비밀 번호와 같은 경우는 노출 위험 때문에 환경 변수를 사용
imap.login(id, pw)

imap.select('INBOX')
resp, data = imap.uid('search', None, 'All')  # 응답 코드와 검색결과 리스트
all_email = data[0].split()
last_email = all_email[-10:]    # 최근 10개의 이메일만 가져오기 
print(resp, len(all_email), len(last_email))

OK 116 1


'(RFC822)'는 이메일 내용을 가져올 때 원본 형식인 RFC822 형식으로 가져오라는 의미입니다. RFC822는 이메일 메시지의 표준 형식을 말합니다.

In [14]:
for mail in reversed(last_email):
    result, data = imap.uid('fetch', mail, '(RFC822)')    # 메일을 읽어온다.
    raw_email = data[0][1]                                # 메일의 내용을 가져온다.
    email_message = email.message_from_bytes(raw_email, policy=policy.default) # 메일의 내용을 파싱한다. 

    print('='*70)
    print('FROM:', email_message['From'])
    print('SENDER:', email_message['Sender'])
    print('TO:', email_message['To'])
    print('DATE:', email_message['Date'])
    print('SUBJECT:', email_message['Subject'])

    print('[CONTENT]')
    message = ''
    if email_message.is_multipart():
        for part in email_message.get_payload():          # 메일의 내용을 가져온다.
            if part.get_content_type() == 'text/plain':   # 메일의 내용 중에서 text/plain 타입만 가져온다.
                bytes = part.get_payload(decode=True)     # 메일의 내용을 가져온다.
                encode = part.get_content_charset()       # 메일의 인코딩을 가져온다.
                message = message + str(bytes, encode)    # 메일의 내용을 문자열로 변환한다.
    #print(message)
    print('='*70)

imap.close()
imap.logout()

FROM: 인프런 학습 소식 <study@inflearn.com>
SENDER: None
TO: kktpsh@naver.com
DATE: Wed, 14 Jun 2023 11:48:30 +0000
SUBJECT: [인프런] 지식공유자 김플님이 새소식을 등록했어요.📫
[CONTENT]


('BYE', [b'Logging out'])